In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Import Libraries**

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# **Import Dataset**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/cleaned.csv')

In [ ]:
df.head()

,Movie_ID,Movie_Title,Movie_Genre,Movie_Language,Movie_Budget,Movie_Popularity,Movie_Release_Date,Movie_Revenue,Movie_Runtime,Movie_Vote,Movie_Vote_Count,Movie_Keywords,Movie_Overview,Movie_Tagline,Movie_Cast,Movie_Director
0,1,Four Rooms,Crime Comedy,en,4000000,22.876230,1995-12-09,4300000,98.0,6.5,530,hotel new year's eve witch bet hotel room,It's Ted the Bellhop's first night on the job....,Twelve outrageous guests. Four scandalous requ...,Tim Roth Antonio Banderas Jennifer Beals Madon...,Allison Anders
1,2,Star Wars,Adventure Action Science Fiction,en,11000000,126.393695,1977-05-25,775398007,121.0,8.1,6624,android galaxy hermit death star lightsaber,Princess Leia is captured and held hostage by ...,"A long time ago in a galaxy far, far away...",Mark Hamill Harrison Ford Carrie Fisher Peter ...,George Lucas
2,3,Finding Nemo,Animation Family,en,94000000,85.688789,2003-05-30,940335536,100.0,7.6,6122,father son relationship harbor underwater fish...,"Nemo, an adventurous young clownfish, is unexp...","There are 3.7 trillion fish in the ocean, they...",Albert Brooks Ellen DeGeneres Alexander Gould ...,Andrew Stanton
3,4,Forrest Gump,Comedy Drama Romance,en,55000000,138.133331,1994-07-06,677945399,142.0,8.2,7927,vietnam veteran hippie mentally disabled runni...,A man with a low IQ has accomplished great thi...,"The world will never be the same, once you've ...",Tom Hanks Robin Wright Gary Sinise Mykelti Wil...,Robert Zemeckis
4,5,American Beauty,Drama,en,15000000,80.878605,1999-09-15,356296601,122.0,7.9,3313,male nudity female nudity adultery midlife cri...,"Lester Burnham, a depressed suburban father in...",Look closer.,Kevin Spacey Annette Bening Thora Birch Wes Be...,Sam Mendes


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3736 entries, 0 to 3735
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Movie_ID            3736 non-null   int64  
 1   Movie_Title         3736 non-null   object 
 2   Movie_Genre         3736 non-null   object 
 3   Movie_Language      3736 non-null   object 
 4   Movie_Budget        3736 non-null   int64  
 5   Movie_Popularity    3736 non-null   float64
 6   Movie_Release_Date  3736 non-null   object 
 7   Movie_Revenue       3736 non-null   int64  
 8   Movie_Runtime       3736 non-null   float64
 9   Movie_Vote          3736 non-null   float64
 10  Movie_Vote_Count    3736 non-null   int64  
 11  Movie_Keywords      3736 non-null   object 
 12  Movie_Overview      3736 non-null   object 
 13  Movie_Tagline       3736 non-null   object 
 14  Movie_Cast          3736 non-null   object 
 15  Movie_Director      3736 non-null   object 
dtypes: flo

In [ ]:
df.shape

(3736, 16)

In [ ]:
df.columns

Index(['Movie_ID', 'Movie_Title', 'Movie_Genre', 'Movie_Language',
       'Movie_Budget', 'Movie_Popularity', 'Movie_Release_Date',
       'Movie_Revenue', 'Movie_Runtime', 'Movie_Vote', 'Movie_Vote_Count',
       'Movie_Keywords', 'Movie_Overview', 'Movie_Tagline', 'Movie_Cast',
       'Movie_Director'],
      dtype='object')

In [ ]:
# Select the relevant columns
df[['Movie_Overview', 'Movie_Vote']]

,Movie_Overview,Movie_Vote
0,It's Ted the Bellhop's first night on the job....,6.5
1,Princess Leia is captured and held hostage by ...,8.1
2,"Nemo, an adventurous young clownfish, is unexp...",7.6
3,A man with a low IQ has accomplished great thi...,8.2
4,"Lester Burnham, a depressed suburban father in...",7.9
...,...,...
3731,When his hard-earned kicks get snatched by a l...,7.5
3732,Sara Gold is a young girl on a quest to save m...,5.3
3733,A squad of soldiers fight in the Korean War's ...,5.8
3734,"Long before he even met Shrek, the notorious f...",6.4


In [ ]:
# Define a function to clean the text data
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()  # Convert text to lowercase
    return text

In [ ]:
# Apply the clean_text function to the 'Movie_Overview' column
df['Movie_Overview'] = df['Movie_Overview'].apply(clean_text)

In [ ]:
# Convert 'Movie_Vote' into binary sentiment labels (1 for positive, 0 for negative)
df['Sentiment'] = df['Movie_Vote'].apply(lambda x: 1 if x >= 5 else 0)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['Movie_Overview'], df['Sentiment'], test_size=0.2, random_state=42)

In [ ]:
# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [ ]:
# Train and evaluate different models

# Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_vec, y_train)
nb_pred = nb_model.predict(X_test_vec)
nb_accuracy = accuracy_score(y_test, nb_pred)
nb_report = classification_report(y_test, nb_pred, zero_division=1)

# Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_vec, y_train)
rf_pred = rf_model.predict(X_test_vec)
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_report = classification_report(y_test, rf_pred, zero_division=1)

# Support Vector Machine
svm_model = SVC(random_state=42)
svm_model.fit(X_train_vec, y_train)
svm_pred = svm_model.predict(X_test_vec)
svm_accuracy = accuracy_score(y_test, svm_pred)
svm_report = classification_report(y_test, svm_pred, zero_division=1)

# Print results
print(f'Naive Bayes Accuracy: {nb_accuracy}')
print(f'Naive Bayes Classification Report:\n{nb_report}')

print(f'Random Forest Accuracy: {rf_accuracy}')
print(f'Random Forest Classification Report:\n{rf_report}')

print(f'SVM Accuracy: {svm_accuracy}')
print(f'SVM Classification Report:\n{svm_report}')


Naive Bayes Accuracy: 0.9264705882352942
Naive Bayes Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.00      0.00        55
           1       0.93      1.00      0.96       693

    accuracy                           0.93       748
   macro avg       0.96      0.50      0.48       748
weighted avg       0.93      0.93      0.89       748

Random Forest Accuracy: 0.9264705882352942
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.00      0.00        55
           1       0.93      1.00      0.96       693

    accuracy                           0.93       748
   macro avg       0.96      0.50      0.48       748
weighted avg       0.93      0.93      0.89       748

SVM Accuracy: 0.9264705882352942
SVM Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.00      0.00        55
           1       0.

# **Get Feature Selection**



In [ ]:
df_features = df[[ 'Movie_Genre', 'Movie_Keywords', 'Movie_Tagline', 'Movie_Cast', 'Movie_Director']].fillna('')

In [ ]:
df_features.shape

(3736, 5)

In [ ]:
df_features

,Movie_Genre,Movie_Keywords,Movie_Tagline,Movie_Cast,Movie_Director
0,Crime Comedy,hotel new year's eve witch bet hotel room,Twelve outrageous guests. Four scandalous requ...,Tim Roth Antonio Banderas Jennifer Beals Madon...,Allison Anders
1,Adventure Action Science Fiction,android galaxy hermit death star lightsaber,"A long time ago in a galaxy far, far away...",Mark Hamill Harrison Ford Carrie Fisher Peter ...,George Lucas
2,Animation Family,father son relationship harbor underwater fish...,"There are 3.7 trillion fish in the ocean, they...",Albert Brooks Ellen DeGeneres Alexander Gould ...,Andrew Stanton
3,Comedy Drama Romance,vietnam veteran hippie mentally disabled runni...,"The world will never be the same, once you've ...",Tom Hanks Robin Wright Gary Sinise Mykelti Wil...,Robert Zemeckis
4,Drama,male nudity female nudity adultery midlife cri...,Look closer.,Kevin Spacey Annette Bening Thora Birch Wes Be...,Sam Mendes
...,...,...,...,...,...
3731,Adventure,blow job cigarette smoking illegal drugs smoki...,They aren't just shoes,Mahershala Ali Kofi Siriboe Christopher Meyer ...,Justin Tipping
3732,Drama,undercover dog animal welfare animal rights or...,She ended up on the wrong side of right.,Allison Paige James Remar Lea Thompson Jayson ...,Alex Ranarivelo
3733,History Drama War Action,"korea fictionalized history operation ""trudy j...",The Odds Were 5000 to 1 … One was all They Nee...,Lee Jung-jae Liam Neeson Lee Beom-soo Jin Se-y...,John H. Lee
3734,Action Adventure Animation Family Fantasy,adventure fairy-tale figure,Live for danger. Fight for justice. Pray for m...,Antonio Banderas Salma Hayek Zach Galifianakis...,Chris Miller


In [ ]:
x = df_features['Movie_Genre'] + '' + df_features['Movie_Keywords'] + '' + df_features['Movie_Tagline'] +''+ df_features['Movie_Cast']+ ''+ df_features['Movie_Director']

In [ ]:
x

0       Crime Comedyhotel new year's eve witch bet hot...
1       Adventure Action Science Fictionandroid galaxy...
2       Animation Familyfather son relationship harbor...
3       Comedy Drama Romancevietnam veteran hippie men...
4       Dramamale nudity female nudity adultery midlif...
                              ...                        
3731    Adventureblow job cigarette smoking illegal dr...
3732    Dramaundercover dog animal welfare animal righ...
3733    History Drama War Actionkorea fictionalized hi...
3734    Action Adventure Animation Family Fantasyadven...
3735    Thriller Dramachristian film sex traffickingSh...
Length: 3736, dtype: object

In [ ]:
x.shape

(3736,)

# Get Feature Text Conversion to Tokens

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
tfidf = TfidfVectorizer()


In [ ]:
x = tfidf.fit_transform(x)


In [ ]:
x.shape

(3736, 23144)

In [ ]:
print(x)

  (0, 882)	0.16399376793991047
  (0, 20873)	0.19580522160421626
  (0, 13250)	0.1418479473495443
  (0, 13012)	0.17477899443518957
  (0, 1811)	0.17989949477206338
  (0, 11153)	0.09897970055040688
  (0, 1608)	0.14296754077088375
  (0, 1044)	0.14078044728365882
  (0, 17755)	0.14296754077088375
  (0, 20800)	0.11455042707933032
  (0, 12495)	0.09036048613230543
  (0, 15157)	0.060577321769720346
  (0, 22538)	0.17477899443518957
  (0, 8162)	0.08519367475403691
  (0, 22484)	0.11031131269716153
  (0, 11222)	0.13608100670933915
  (0, 20286)	0.10235662073927458
  (0, 15235)	0.07121741273261348
  (0, 5236)	0.11893459322907225
  (0, 8023)	0.11700512841834275
  (0, 10000)	0.19795940110081375
  (0, 10720)	0.1502023992069678
  (0, 1959)	0.19580522160421626
  (0, 12692)	0.15285276512076806
  (0, 15238)	0.08384609896687899
  :	:
  (3734, 379)	0.0847408618864912
  (3734, 1608)	0.18880589898275651
  (3734, 1044)	0.1859175779709501
  (3734, 8162)	0.3375256004862958
  (3735, 2432)	0.28444824987693956
  (3735,

# **Get Similarity Score using Cosine Similarity**


In [ ]:
 from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
Similarity_Score = cosine_similarity(x)


In [ ]:
Similarity_Score

array([[1.        , 0.01382658, 0.03604693, ..., 0.00676268, 0.08192172,
        0.        ],
       [0.01382658, 1.        , 0.00808918, ..., 0.        , 0.01172742,
        0.        ],
       [0.03604693, 0.00808918, 1.        , ..., 0.01954575, 0.04699168,
        0.        ],
       ...,
       [0.00676268, 0.        , 0.01954575, ..., 1.        , 0.        ,
        0.005821  ],
       [0.08192172, 0.01172742, 0.04699168, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.005821  , 0.        ,
        1.        ]])

In [ ]:
Similarity_Score.shape

(3736, 3736)

**Get Movie Name as Input from User and Validate for Closest Spelling**

In [ ]:
Favourite_Movie_Name = input(' Enter your favourite movie name :')

 Enter your favourite movie name :harry


In [ ]:
All_Movies_Title_List = df['Movie_Title'].tolist()

In [ ]:
import difflib

In [ ]:
Movie_Recommendation = difflib.get_close_matches (Favourite_Movie_Name, All_Movies_Title_List)
print(Movie_Recommendation)

['Charly']


In [ ]:
Close_Match = Movie_Recommendation[0]
print (Close_Match)

Charly


In [ ]:

Index_of_Close_Match_Movie = df [df.Movie_Title == Close_Match]['Movie_ID'].values[0]
print(Index_of_Close_Match_Movie)


2996


In [ ]:
# getting a list of similar movies

Recommendation_Score = list(enumerate(Similarity_Score[Index_of_Close_Match_Movie]))
print (Recommendation_Score)




[(0, 0.0), (1, 0.004920287286691915), (2, 0.07759724929156522), (3, 0.0037205877838630756), (4, 0.0), (5, 0.004268616667863646), (6, 0.035517139582648065), (7, 0.01614241063637173), (8, 0.0), (9, 0.005838915157464608), (10, 0.00448429369375106), (11, 0.004775763542382202), (12, 0.0), (13, 0.0), (14, 0.003957445030427018), (15, 0.003748536376701837), (16, 0.0), (17, 0.013166230868043135), (18, 0.004319033015105753), (19, 0.0), (20, 0.004875828526644268), (21, 0.02358065480004672), (22, 0.004485158314581468), (23, 0.0), (24, 0.0036381109317697243), (25, 0.03263695719882572), (26, 0.009905471789256591), (27, 0.0), (28, 0.004181523025978563), (29, 0.01680852478177742), (30, 0.0040755156933337065), (31, 0.008975975973861206), (32, 0.0038548021124813155), (33, 0.02718645125979511), (34, 0.0), (35, 0.0), (36, 0.0), (37, 0.005992186805466483), (38, 0.014859549315144313), (39, 0.010534854016171684), (40, 0.0), (41, 0.0), (42, 0.008268344441571773), (43, 0.0), (44, 0.005092219815047479), (45, 0.

In [ ]:
 len(Recommendation_Score)



3736

# **Get All Movies Sorted Based on Recommendation Score for your Favourite Movie**


In [ ]:
#sorting the movies based on their similarity score

Sorted_Similar_Movies = sorted(Recommendation_Score, key = lambda x:x[1], reverse=True)
print (Sorted_Similar_Movies)


[(2996, 1.0000000000000002), (3108, 0.15458309447000693), (910, 0.1429941879538134), (2287, 0.13352123872959556), (3652, 0.12893389146541778), (2565, 0.12426875214049662), (2626, 0.11621291338219178), (3140, 0.11426010685062842), (3507, 0.10741881939884965), (3054, 0.09840953143447427), (3321, 0.09674092090406851), (2952, 0.09577937625394546), (3319, 0.09451332743333611), (3022, 0.09386808866918477), (3692, 0.09221551409701169), (2723, 0.09150527597746182), (2445, 0.09031195826065214), (2991, 0.08654031638120063), (2203, 0.08509129646819479), (317, 0.08150502097378559), (2539, 0.08105302586972365), (940, 0.07834656660932358), (948, 0.07827068621019231), (2, 0.07759724929156522), (404, 0.07319002725588836), (501, 0.0712929875458319), (3677, 0.07108934190441538), (3684, 0.07076275894304075), (2431, 0.07059551034052788), (2874, 0.0694711497528241), (1879, 0.06932371166118441), (402, 0.06695739339377707), (2802, 0.0666009318312126), (2164, 0.06407803086539136), (478, 0.06354545981822739), 

In [ ]:
# print the name of similar movies based on the index

print('Top 30 Movies Suggested for You :\ n ')

i=1

for movie in Sorted_Similar_Movies:
   index = movie[0]
   title_from_index = df [df.index==index]['Movie_Title'].values[0]
   if (i<31):
    print(i, '.',title_from_index)
    i+=1

Top 30 Movies Suggested for You :\ n 
1 . Drive
2 . Only God Forgives
3 . No Country for Old Men
4 . Bronson
5 . The Neon Demon
6 . Wall Street: Money Never Sleeps
7 . The Muse
8 . Gangster Squad
9 . A Most Violent Year
10 . Red Tails
11 . Kung Fu Panda 3
12 . Larry Crowne
13 . Detention of the Dead
14 . Argo
15 . The Infiltrator
16 . Never Let Me Go
17 . An Education
18 . The Great Gatsby
19 . The Brothers McMullen
20 . Evil Dead II
21 . Bodyguards and Assassins
22 . Smokin' Aces
23 . Half Nelson
24 . Finding Nemo
25 . Irreversible
26 . The Net
27 . The Big Short
28 . Concussion
29 . The Little Vampire
30 . Crazy, Stupid, Love.


# **Top 10 Movies Recommended Based on Your Favorite Movie**


In [ ]:
import difflib

Movie_Name = input('Enter your favorite movie name: ')

list_of_all_titles = df['Movie_Title'].tolist()

# Find close matches to the input movie name
close_matches = difflib.get_close_matches(Movie_Name, list_of_all_titles)

if close_matches:
    closest_match = close_matches[0]  # Get the closest match
    Index_of_Movie = df[df.Movie_Title == closest_match]['Movie_ID'].values[0]

    Recommendation_Score = list(enumerate(Similarity_Score[Index_of_Movie]))

    sorted_similar_movies = sorted(Recommendation_Score, key=lambda x: x[1], reverse=True)

    print('Top 10 Movies suggested for you: \n')

    i = 1

    for movie in sorted_similar_movies:
        index = movie[0]
        if index < len(df):
            title_from_index = df[df.Movie_ID == index]['Movie_Title'].values[0]
            print(i, '.', title_from_index)
            i += 1
        else:
            print("Invalid index:", index)

        if i > 10:
            break
else:
    print('No close matches found for the entered movie name.')


Enter your favorite movie name: harry potter
Top 10 Movies suggested for you: 

1 . Party Monster
2 . The Blair Witch Project
3 . The Doors
4 . Moby Dick
5 . Alfie
6 . Run, Fatboy, Run
7 . The Replacements
8 . A Hard Day's Night
9 . Darkness Falls
10 . The Animal
